**Reading Predicted Sale**


In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from pulp import *
import itertools

In [ ]:
predict = pd.read_csv('meta_class.csv')

In [ ]:
predict.head()

,miladi-flight-date,hour,origin,destination,ticket_serial,ticket_total_price
0,2018-10-28,5,Shiraz,Tehran,1,2742000.0
1,2018-10-28,6,Tabriz,Tehran,2,2041000.0
2,2018-10-29,2,Tehran,Mashhad,1,2769500.0
3,2018-10-29,2,Urmia,Tehran,1,1882000.0
4,2018-10-29,3,Tehran,Sanandaj,1,2412000.0


In [ ]:
predict.shape

(487100, 6)

In [ ]:
predict.tail()

,miladi-flight-date,hour,origin,destination,ticket_serial,ticket_total_price
487095,2023-03-20,5,tehran,mashhad,1,10175000.0
487096,2023-03-20,6,Tehran,Dezfool,2,9230000.0
487097,2023-03-22,5,Tehran,Mashhad,1,10175000.0
487098,2023-03-31,4,kish island,tehran,4,12580250.0
487099,2023-04-01,4,kish island,tehran,2,13592000.0


**Needed Cols: date, span, origin, destination, ticket**

In [ ]:
dict_col = {'miladi-flight-date':'date',
            'hour':'span',
            'origin':'origin',
            'destination':'destination',
            'ticket_serial':'ticket'}
predict.rename(columns=dict_col, inplace=True)

**select requested span**

In [ ]:
print(f"min date: {predict['date'].min()}\nmax date: {predict['date'].max()}")

min date: 2018-10-28
max date: 2023-04-01


In [ ]:
start = '2020-01-01'
# start = input('Enter a start date in YYYY-MM-DD format: ')

In [ ]:
end = '2020-01-08'
# end = input('Enter a end date in YYYY-MM-DD format: ')

In [ ]:
slice = predict[(predict['date'] >= start) & (predict['date'] <= end)]

In [ ]:
slice

,date,span,origin,destination,ticket,ticket_total_price
121226,2020-01-01,1,Mashhad,Tehran,3,2.262000e+06
121227,2020-01-01,2,Ahvaz,Tehran,13,3.664308e+06
121228,2020-01-01,2,Bam,Tehran,4,3.679500e+06
121229,2020-01-01,2,Isfahan,Tehran,10,3.282700e+06
121230,2020-01-01,2,Kerman,Tehran,7,4.311000e+06
...,...,...,...,...,...,...
124439,2020-01-08,6,Mashhad,Tehran,15,2.111333e+06
124440,2020-01-08,6,Tabriz,Tehran,23,2.327261e+06
124441,2020-01-08,6,Tehran,Kerman,1,4.071000e+06
124442,2020-01-08,6,Tehran,Mashhad,7,3.206143e+06


**preprocsess dataset**

In [ ]:
slice = slice.sort_values(['date','span'])

In [ ]:
slice['concat'] = slice['date'].astype(str) + slice['span'].astype(str)

In [ ]:
slice['time'] = pd.factorize(slice['concat'])[0]

In [ ]:
slice

,date,span,origin,destination,ticket,ticket_total_price,concat,time
121226,2020-01-01,1,Mashhad,Tehran,3,2.262000e+06,2020-01-011,0
121227,2020-01-01,2,Ahvaz,Tehran,13,3.664308e+06,2020-01-012,1
121228,2020-01-01,2,Bam,Tehran,4,3.679500e+06,2020-01-012,1
121229,2020-01-01,2,Isfahan,Tehran,10,3.282700e+06,2020-01-012,1
121230,2020-01-01,2,Kerman,Tehran,7,4.311000e+06,2020-01-012,1
...,...,...,...,...,...,...,...,...
124439,2020-01-08,6,Mashhad,Tehran,15,2.111333e+06,2020-01-086,43
124440,2020-01-08,6,Tabriz,Tehran,23,2.327261e+06,2020-01-086,43
124441,2020-01-08,6,Tehran,Kerman,1,4.071000e+06,2020-01-086,43
124442,2020-01-08,6,Tehran,Mashhad,7,3.206143e+06,2020-01-086,43


In [ ]:
slice = slice.groupby(['time','origin','destination']).agg({'ticket':sum,'date':min,'span':min}).reset_index()

**initialize class**

In [ ]:
model = LpProblem('Maximize_Sale', LpMaximize)

In [ ]:
times = predict['time'].unique().tolist()

In [ ]:
cities = np.unique(predict[['origin', 'destination']].values).tolist()

**creat all combinations**

In [ ]:
cities_combination = list(itertools.combinations_with_replacement(cities, 2))

In [ ]:
all_combination = list(itertools.product(cities_combination, times))

In [ ]:
df = pd.DataFrame(all_combination,columns=['path','time'])

In [ ]:
df[['origin','destination']] = pd.DataFrame(df["path"].to_list())

In [ ]:
merge = df.merge(predict, on=['origin','destination','time'], how='left')

In [ ]:
merge['ticket'] = merge['ticket'].fillna(0)

**parameter of our modeling**

In [ ]:
weight = merge[['time','origin', 'destination','ticket']]

In [ ]:
weight_dict = weight.to_dict('records')

**define decision variables**

In [ ]:
x = LpVariable.dicts('x',[(i,j,t) for i in cities for j in cities for t in times], cat = 'Binary')

**objective function**

In [ ]:
model += lpSum([weight[(weight['time']==t) & (weight['origin']==i) & (weight['destination']==j)].ticket.values
                * x[(i,j,t)] for i in cities for j in cities for t in times])

**define constraints**

1 - connected route

In [ ]:
for t in times[:-1]:
  for j in cities:
    model += lpSum([x[i,j,t] for i in cities]) == lpSum([x[j,k,t+1] for k in cities])

2 - only 1 travel for each span

In [ ]:
for t in times:
  model += lpSum([x[i,j,t] for i in cities for j in cities]) == 1

3 - self loop is not allowed

In [ ]:
for t in times:
  model += lpSum([x[i,i,t] for i in cities]) == 0

**solve model**

In [ ]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/06/v5w2n0q54sz8wd7kwgqc3j280000gn/T/969682782b0047219dcd7808907b2f12-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/06/v5w2n0q54sz8wd7kwgqc3j280000gn/T/969682782b0047219dcd7808907b2f12-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2373 COLUMNS
At line 709712 RHS
At line 712081 BOUNDS
At line 831058 ENDATA
Problem MODEL has 2368 rows, 118976 columns and 468252 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1250 - 0.46 seconds
Cgl0002I 2288 variables fixed
Cgl0004I processed model has 2280 rows, 109508 columns (109508 integer (109508 of which binary)) and 433704 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial stat

1

In [ ]:
print('status:', LpStatus[model.status])

status: Optimal


In [ ]:
model.variables()

[x_('Abadan',_'Abadan',_0),
 x_('Abadan',_'Abadan',_1),
 x_('Abadan',_'Abadan',_10),
 x_('Abadan',_'Abadan',_11),
 x_('Abadan',_'Abadan',_12),
 x_('Abadan',_'Abadan',_13),
 x_('Abadan',_'Abadan',_14),
 x_('Abadan',_'Abadan',_15),
 x_('Abadan',_'Abadan',_16),
 x_('Abadan',_'Abadan',_17),
 x_('Abadan',_'Abadan',_18),
 x_('Abadan',_'Abadan',_19),
 x_('Abadan',_'Abadan',_2),
 x_('Abadan',_'Abadan',_20),
 x_('Abadan',_'Abadan',_21),
 x_('Abadan',_'Abadan',_22),
 x_('Abadan',_'Abadan',_23),
 x_('Abadan',_'Abadan',_24),
 x_('Abadan',_'Abadan',_25),
 x_('Abadan',_'Abadan',_26),
 x_('Abadan',_'Abadan',_27),
 x_('Abadan',_'Abadan',_28),
 x_('Abadan',_'Abadan',_29),
 x_('Abadan',_'Abadan',_3),
 x_('Abadan',_'Abadan',_30),
 x_('Abadan',_'Abadan',_31),
 x_('Abadan',_'Abadan',_32),
 x_('Abadan',_'Abadan',_33),
 x_('Abadan',_'Abadan',_34),
 x_('Abadan',_'Abadan',_35),
 x_('Abadan',_'Abadan',_36),
 x_('Abadan',_'Abadan',_37),
 x_('Abadan',_'Abadan',_38),
 x_('Abadan',_'Abadan',_39),
 x_('Abadan',_'Aba

In [ ]:
for v in model.variables():
    if v.varValue == 1:
        print(v.name)

x_('Ahvaz',_'Tehran',_16)
x_('Ahvaz',_'Tehran',_35)
x_('Ahvaz',_'Tehran',_5)
x_('Bandar_e_abbas',_'Tehran',_18)
x_('Bandar_e_abbas',_'Tehran',_32)
x_('Isfahan',_'Mashhad',_20)
x_('Kerman',_'Mashhad',_7)
x_('Kish_island',_'Mashhad',_25)
x_('Kish_island',_'Shiraz',_0)
x_('Kish_island',_'Tehran',_30)
x_('Mashhad',_'Tehran',_12)
x_('Mashhad',_'Tehran',_14)
x_('Mashhad',_'Tehran',_21)
x_('Mashhad',_'Tehran',_26)
x_('Mashhad',_'Tehran',_28)
x_('Mashhad',_'Tehran',_3)
x_('Mashhad',_'Tehran',_37)
x_('Mashhad',_'Tehran',_39)
x_('Mashhad',_'Tehran',_41)
x_('Mashhad',_'Tehran',_43)
x_('Mashhad',_'Tehran',_8)
x_('Shiraz',_'Tehran',_1)
x_('Shiraz',_'Tehran',_10)
x_('Shiraz',_'Tehran',_23)
x_('Tehran',_'Ahvaz',_15)
x_('Tehran',_'Ahvaz',_4)
x_('Tehran',_'Bandar_e_abbas',_17)
x_('Tehran',_'Bandar_e_abbas',_31)
x_('Tehran',_'Isfahan',_19)
x_('Tehran',_'Kerman',_6)
x_('Tehran',_'Kish_island',_24)
x_('Tehran',_'Kish_island',_29)
x_('Tehran',_'Mashhad',_11)
x_('Tehran',_'Mashhad',_13)
x_('Tehran',_'Mashha

In [ ]:
output = []
for origin, destination, time in x:
    var_output = {
        'origin': origin,
        'destination': destination,
        'time': time,
        'result': x[(origin, destination, time)].varValue,
    }
    output.append(var_output)

In [ ]:
output_df = pd.DataFrame.from_records(output).sort_values(['time'])

In [ ]:
output_df

,origin,destination,time,result
0,Abadan,Abadan,0,0.0
7744,Asalooye,Jiroft,0,0.0
109384,Urmia,Shiraz,0,0.0
69124,Maraghe,Bojnourd,0,0.0
69168,Maraghe,Dezfool,0,0.0
...,...,...,...,...
77439,Pars abad,Sirjan,43,0.0
77395,Pars abad,Shiraz,43,0.0
77351,Pars abad,Shahr-e kord,43,0.0
77263,Pars abad,Sanandaj,43,0.0


In [ ]:
final = output_df[output_df['result']==1].reset_index()

In [ ]:
final

,index,origin,destination,time,result
0,61336,Kish island,Shiraz,0,1.0
1,98121,Shiraz,Tehran,1,1.0
2,106614,Tehran,Mashhad,2,1.0
3,72955,Mashhad,Tehran,3,1.0
4,105296,Tehran,Ahvaz,4,1.0
5,4317,Ahvaz,Tehran,5,1.0
6,106222,Tehran,Kerman,6,1.0
7,51707,Kerman,Mashhad,7,1.0
8,72960,Mashhad,Tehran,8,1.0
9,107105,Tehran,Shiraz,9,1.0


In [ ]:
merge = final.merge(slice, on = ['origin', 'destination', 'time'], how = 'left')

In [ ]:
schedule = merge[['date','span','time','origin','destination','ticket']]

In [ ]:
schedule

,date,span,time,origin,destination,ticket
0,NaN,NaN,0,Kish island,Shiraz,NaN
1,2020-01-01,2.0,1,Shiraz,Tehran,43.0
2,2020-01-01,3.0,2,Tehran,Mashhad,29.0
3,2020-01-01,4.0,3,Mashhad,Tehran,49.0
4,2020-01-01,5.0,4,Tehran,Ahvaz,64.0
5,2020-01-01,6.0,5,Ahvaz,Tehran,69.0
6,NaN,NaN,6,Tehran,Kerman,NaN
7,2020-01-02,3.0,7,Kerman,Mashhad,14.0
8,2020-01-02,4.0,8,Mashhad,Tehran,51.0
9,2020-01-02,5.0,9,Tehran,Shiraz,40.0


In [ ]:
print('objective=', value(model.objective))

objective= 1250.0
